бейзлайн
- первая часть (нахождения новостей с упоминанием компаний)
    - использовать только заголовки (датасет https://github.com/yutkin/Lenta.Ru-News-Dataset)
    - ~находить компании в заголовках через заранее заготовленный список компаний с Московской биржи~
    - сначала попробовать только на Яндексе
- вторая часть (проверять, как конкретная новость повлияла на акции компании)
    - смотреть, изменилась ли цена (на 10%, Яндекс из-за закона Горелкина падал на 15%) на фиксированный процент в тот же день (далее можно попробовать 24 рабочих часа после новости)
    - таким образом сделаем датасет для второй задачи (вида "текст новости - компания - таргет (цена понизилась / осталась прежней / повысилась)")
    - обучить на трейне такого датасета модель, которая получает на вход эмбеддинг предложения и пытается предсказать влияние на акции (в какой-то степени это будет sentiment analysis), вначале можно простенькую модель из нескольких слоев с релу, в конце софтмакс на 3 класса



### Сейчас:
- ~найти, откуда взять данные по бирже~ -- yfinance
- ~посмотреть период закона Горелкина (27.07.19) (нужно проверить наличие тех же проблем, что внизу этой ячейки)~ -- вроде по графикам таких проблем нет
- ~сделать функцию, которая к каждой новости покажет изменение цены в процентах~
    - считать изменение к прошлому дню, если вчерашнее или сегодняшнее значение недоступно, то NaN
- ~засунуть эти значения к соответствующим новостям~
- ~посчитать таргеты (изменения вверх/вниз больше х% (1/-1) или 0)~
- добавить других компаний
    - где-то получить лист русских компаний, который есть в yahoo finance
- обучить модель (предобученную на sentiment analysis, добавить пару слоев сверху и выход из 3 логитов, дальше софтмакс и получаем вероятность класса)

### Проверки:
- что случилось после новостей о законе Горелкина

### Проблемы:
- ~Проблемы с данными от pandas_datareader, в примере ниже слишком часто акции торгуются в нуле
```mr = pandas_datareader.data.DataReader('YNDX', 'moex', start='2019-01-01', end='2019-10-31')
mr.plot(subplots=True, figsize=(5, 10))```~ -- больше не использую pandas datareader, использую yfinance
- ~из-за закона Горелкина (27.07.19) акции упали с 2520 до 2441, но без новостей падают еще больше (27.05.19, 2404 -> 2280)~ -- проблема из-за pandas datareader, больше не актуально

In [1]:
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np
import tqdm
import dateutil
import datetime
import yfinance as yf

- взять интерфакс, не только российские компании
- посмтреть глазами, насколько разумны -1 и 1 (и из-за этого возможно поменять параметры)
- достать данные за более короткий срок
- обучить классификатор, ненулевых таргетов нужно 1000 (для предобученных можно меньше)
- разница -15 минут и 4 часа, смотреть разницу между ними, если больше 5-10%, то ненулвой таргет (также можно сравнить минимум-максимум за этот промежуток)

In [2]:
# !head /home/student/large_files_data/interfax_20160101_20191015.csv -n 2 > /home/student/large_files_data/interfax_small.csv
# !head /home/student/large_files_data/interfax_20160101_20191015.csv -n 1


In [3]:
interfax_news_table = pd.read_csv(
    '/home/student/large_files_data/interfax_20160101_20191015.csv',
#     sep=',',
    na_values='-',
    escapechar='\\',
    index_col=False,
    usecols=['date', 'title', 'text']
)

In [4]:
interfax_news_table.head()

,date,title,text
0,2019-09-09 14:09:00,Верховный суд в октябре рассмотрит жалобу на п...,Москва. 9 сентября. INTERFAX.RU - Верховный су...
1,2019-09-10 14:12:00,Суд Петербурга отменил выдворение блогера-оппо...,Москва. 10 сентября. INTERFAX.RU - Петербургск...
2,2019-09-11 14:20:00,Евро опустился ниже 72 рублей впервые со 2 авг...,Москва. 11 сентября. INTERFAX.RU - Евро в ходе...
3,2019-09-12 15:30:00,"ЕЦБ снизил ставку по депозитам до минус 0,5%",Москва. 12 сентября. INTERFAX.RU - Европейский...
4,2019-09-13 13:40:00,Москва негативно отнеслась к словам Зеленского...,Москва. 13 сентября. INTERFAX.RU - В Кремле не...


In [5]:
companies_to_listing_names = {
    'Яндекс': 'YNDX',
    'Роснефть': 'ojscy',
    'Лукойл': 'lukoy',
    'Сбербанк': 'SBER.ME',
    'Русал': 'RUAL.ME',
    'Газпром': 'OGZPY',
    'ВТБ': 'VTBR.ME',
    'Магнит': 'MGNT.ME',
    'Транснефть': 'TRNFP.ME',
    'X5 Retail Group': 'FIVE.ME',
    'Сургутнефтегаз': 'SGGD.VI',
    
    'Apple': 'AAPL',
    'Microsoft': 'MSFT',
    'Google': 'GOOG',
    'Facebook': 'FB',
    'Amazon': 'AMZN',
    'Netflix': 'NFLX',
    
}

listing_names_to_companies = {value: key for (key, value) in companies_to_listing_names.items()}

In [6]:
# for name in companies_to_listing_names.keys():
#     company_df = yf.Ticker(
#         companies_to_listing_names[name]
#     )
#     hist = company_df.history(
#         period="max",
#         interval='1d',
# #         start='2016-05-01',
# #         end='2016-05-02'
#     )

In [7]:
news_dataset_path = '/home/student/large_files_data/lenta-ru-news.csv'

news_table = pd.read_csv(
    news_dataset_path,
#     nrows=10000,
    usecols=['title', 'url']
)

# Extract date
news_table['date'] = pd.to_datetime(news_table['url'], format='%y/%m/%d', exact=False)

# Drop url column (it was needed only for date)
news_table = news_table.drop(['url'], axis=1)

# Leave only companies-related news
companies_names = list(companies_to_listing_names.keys())
company_news_table_parts = []
for name in companies_names:
    company_df = yf.Ticker(companies_to_listing_names[name])
    hist = company_df.history(period="max")
    close_change = pd.DataFrame(hist['Close'].pct_change())
    close_change.rename(columns={'Close': 'price_change'}, errors="raise", inplace=True)
    company_news_table = news_table.join(close_change, on='date')

    company_news_table['company'] = name
    
    company_news_table_parts.append(company_news_table[company_news_table.title.str.contains(name, case=False)])

dataset = pd.concat(company_news_table_parts).reset_index(drop=True)
dataset.dropna(axis=0, subset=['price_change'], inplace=True)

dataset.head()

,title,date,price_change,company
151,"""Яндекс"" подорожал в первый день торгов на 55 ...",2011-05-25,-0.098867,Яндекс
152,"""Яндекс"" продал дополнительный пакет акций",2011-05-27,-0.009203,Яндекс
154,"""Рамблер"" будет использовать поиск ""Яндекса""",2011-06-23,0.014979,Яндекс
155,"""Яндекс"" не нашел различий между мужчинами и ж...",2011-06-28,0.045846,Яндекс
157,"В мобильных ""Яндекс.Картах"" появились панорамы...",2011-07-11,-0.050626,Яндекс


In [8]:
len(dataset)

14163

In [9]:
def calc_target(dataset, threshold=0.1):
    return \
        (dataset['price_change'] > threshold) * 1 + \
        (dataset['price_change'] < -threshold) * -1

dataset['target'] = calc_target(dataset)
new_dataset = dataset.drop(['price_change', 'date'], axis=1)

len(new_dataset[new_dataset['target'] != 0]), len(dataset)

(148, 14163)

Получается, что всего 7078 объектов, из них с ненулевым таргетом --- 105

In [14]:
dataset[dataset['target'] != 0].drop(columns=['date', 'price_change'])

,title,company,target
176,"Росреестр решил создать конкурента картам ""Янд...",Яндекс,-1
177,"В веб-версии ""Яндекс.Карт"" появились отметки о...",Яндекс,1
323,«Яндекс.Деньги» сняли ограничения на пополнени...,Яндекс,-1
324,Квартальный отчет обвалил акции «Яндекса»,Яндекс,-1
440,В «Яндекс.Новостях» появились ссылки на западн...,Яндекс,-1
543,«Яндекс» исследовал изменения туристических пр...,Яндекс,1
634,«Яндекс» и Uber объединили бизнес по заказу та...,Яндекс,1
635,ФАС увидела угрозу конкуренции в объединении U...,Яндекс,1
636,В сети пошутили над «униженным Uber» и транспо...,Яндекс,1
637,В Uber оценили долю объединенной с «Яндексом» ...,Яндекс,1


In [15]:
from deeppavlov import train_model

ModuleNotFoundError: No module named 'deeppavlov'